In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds


In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info = True, as_supervised = True)

## Data preprocessing

In [3]:
mnist_train , mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1*mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples,tf.int64)

num_test_samples =  mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

## functionn that scale the image value between 0 and 1
## for  dataset.map(*function*) 
def scale(image,lable):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image,lable

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))





## Model

In [14]:
input_size = 784
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size,activation='softmax')
                            ])

In [15]:
model.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [16]:
NUM_EPOCHS = 5

model.fit(train_data, epochs= NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose = 2)

Epoch 1/5
540/540 - 17s - loss: 0.2592 - accuracy: 0.9234 - val_loss: 0.1310 - val_accuracy: 0.9593 - 17s/epoch - 31ms/step
Epoch 2/5
540/540 - 18s - loss: 0.1023 - accuracy: 0.9688 - val_loss: 0.0941 - val_accuracy: 0.9725 - 18s/epoch - 34ms/step
Epoch 3/5
540/540 - 12s - loss: 0.0699 - accuracy: 0.9782 - val_loss: 0.0664 - val_accuracy: 0.9785 - 12s/epoch - 23ms/step
Epoch 4/5
540/540 - 13s - loss: 0.0515 - accuracy: 0.9837 - val_loss: 0.0585 - val_accuracy: 0.9833 - 13s/epoch - 25ms/step
Epoch 5/5
540/540 - 16s - loss: 0.0440 - accuracy: 0.9865 - val_loss: 0.0349 - val_accuracy: 0.9878 - 16s/epoch - 29ms/step


## Test The Model

In [17]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 1s 548ms/step - loss: 0.0757 - accuracy: 0.9772


In [20]:
print('Test loss : {0:.2f}; Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100))

Test loss : 0.08; Test accuracy: 97.72%
